In [1]:
import nbimporter
from librerias_funciones import *

In [2]:
#Obtenemos todos los datos
fecha_scrapeo = datetime.now().date()

def scrapear_linkedin(empleo):

    while True:
        data = []
        sleep(4)

        # Obtenemos todas las ofertas de la pagina actual
        while True:
            ofertas_visibles = driver.find_elements(By.CSS_SELECTOR, value = 'div[data-view-name = "job-card"]')
            if not ofertas_visibles:
                return next
            
            driver.execute_script("arguments[0].scrollIntoView();", ofertas_visibles[-1])
            sleep(1)
            nuevas_ofertas_visibles = driver.find_elements(By.CSS_SELECTOR, value = 'div[data-view-name = "job-card"]')
            if len(nuevas_ofertas_visibles) != len(ofertas_visibles):
                next
            else:
                break

        #Extraccion de datos de cada oferta
        for oferta in ofertas_visibles:
            driver.execute_script("arguments[0].scrollIntoView();", oferta)
            sleep(2)
            oferta.click()
            sleep(2)
            datos = driver.find_element(By.CSS_SELECTOR, value = 'div[class = "job-details-jobs-unified-top-card__primary-description-without-tagline mb2"]').text
            sleep(2)

            # try except para no perder datos, ya que si no se consigue realizar la accion se volvera a la oferta mas tarde
            try:
                titulo, empresa = oferta.text.split('\n')[:2]
            except:
                ofertas_visibles.extend([oferta])
                continue

            try:
                ubicacion, fecha, solicitudes = datos.split('·')[1:]
            except:
                ubicacion, fecha, solicitudes = np.nan, np.nan, np.nan

            descripcion = driver.find_element(By.ID, value = 'job-details').text.replace('\n', '')

            jornada_presencialidad = driver.find_elements(By.CLASS_NAME, value = 'job-details-jobs-unified-top-card__job-insight')[0].text
            try:
                jornada = [jornada for jornada in tipos_empleos if jornada in jornada_presencialidad][0]
            except:
                jornada = np.nan
                next
            try:
                presencialidad = [presencialidad for presencialidad in tipos_presencialidad if presencialidad in jornada_presencialidad][0]
            except:
                presencialidad = np.nan
                next
            #mostrar todas las aptitudes
            while True:
                try:
                    #Tratamos de buscar si hay aptitudes
                    try:
                        [enlace_aptitudes for enlace_aptitudes in driver.find_elements(By.CLASS_NAME, value = 'app-aware-link') if 'Aptitudes' in enlace_aptitudes.text][0].click()
                        sleep(2)

                        [mostrar_aptitudes for mostrar_aptitudes in driver.find_elements(By.CLASS_NAME, value = 'artdeco-button__text') if mostrar_aptitudes.text == 'Mostrar todas las aptitudes'][0].click()
                        sleep(2)

                        try:
                            caja_aptitudes = driver.find_element(By.CLASS_NAME, value = 'job-details-skill-match-status-list')

                        except NoSuchElementException:
                            #Salir de la caja de aptitudes
                            [boton for boton in driver.find_elements(By.CLASS_NAME, value = 'artdeco-button__text') if boton.text == 'Done'][0].click()
                            continue

                        sleep(4)
                        herramientas = [aptitud.text.replace('\nAñadir', '') for aptitud in caja_aptitudes.find_elements(By.TAG_NAME, value = 'li')]

                        sleep(2)

                        #Salir de la caja de aptitudes
                        [boton for boton in driver.find_elements(By.CLASS_NAME, value = 'artdeco-button__text') if boton.text == 'Done'][0].click()

                        break
                    except StaleElementReferenceException:
                        next
                except:

                    heramientas = np.nan
                    break

            url = driver.current_url

            data.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, jornada, presencialidad, solicitudes, fecha_scrapeo, url, portal_busqueda])

        #Guardamos cada vez que termina de sacrapear una pagina
        guardar_datos(portal_busqueda = portal_busqueda, nombres_columnas = nombres_columnas, data = data)

        pagina_actual = driver.find_element(By.CSS_SELECTOR, 'li[class = "artdeco-pagination__indicator artdeco-pagination__indicator--number active selected ember-view"]')

        try:
            pagina_actual.find_element(By.XPATH, ".//following-sibling::li").click()

        except:
            driver.quit()
            print(f'{empleo} - Scrapeo completado')
            break

In [3]:
url = 'https://www.linkedin.com/'
usuario = 'bruno.octavio.tomas@gmail.com'
password = 'password123'

empleos_castellano = [
    "Desarrollador de software",
    "Ingeniero de desarrollo",
    "Analista de sistemas",
    "Desarrollador web",
    "Ingeniero DevOps",
    "Administrador de bases de datos",
    "Científico de datos",
    "Ingeniero de aprendizaje automático",
    "Desarrollador de aplicaciones",
    "Analista de seguridad informática",
    "Ingeniero de redes",
    "Ingeniero de sistemas",
    "Desarrollador de juegos",
    "Analista de negocios de TI",
    "Arquitecto de software",
    "Ingeniero de pruebas de software",
    "Desarrollador de interfaces de usuario",
    "Ingeniero de automatización",
    "Especialista en análisis de rendimiento",
    "Ingeniero de realidad extendida"
]

empleos_ingles = [
    "Software Developer",
    "Development Engineer",
    "Systems Analyst",
    "Web Developer",
    "DevOps Engineer",
    "Database Administrator",
    "Data Scientist",
    "Machine Learning Engineer",
    "Applications Developer",
    "IT Security Analyst",
    "Network Engineer",
    "Systems Engineer",
    "Game Developer",
    "IT Business Analyst",
    "Software Architect",
    "Software Test Engineer",
    "UI/UX Developer",
    "Automation Engineer",
    "Performance Analyst",
    "Extended Reality Engineer"
]

nombres_columnas = ['titulo', 'empresa', 'fecha', 'herramientas', 'descripcion',
       'ubicacion', 'jornada', 'presencialidad', 'solicitudes',
       'fecha_scrapeo', 'url', 'portal']
portal_busqueda = 'linkedin'


opciones_fechas_publicacion = ['Últimas 24 horas', 'Semana pasada']

In [4]:
'''
# Asignar la función de manejo a los cambios en la selección
selector_variable = crear_widget_seleccion(opciones_fechas_publicacion)
selector_variable.observe(handle_variable_selection, names='value')

# Mostrar el widget
display(selector_variable)
'''

"\n# Asignar la función de manejo a los cambios en la selección\nselector_variable = crear_widget_seleccion(opciones_fechas_publicacion)\nselector_variable.observe(handle_variable_selection, names='value')\n\n# Mostrar el widget\ndisplay(selector_variable)\n"

In [5]:
intervalo_temporal_busqueda = 'Últimas 24 horas'

In [ ]:
for empleo in empleos_castellano:
    try:
        driver = webdriver.Chrome()
        driver.get(url)
        sleep(1)
        driver.maximize_window()

        #Iniciamos sesion
        driver.find_element(By.ID, value = 'session_key').send_keys(usuario)
        sleep(1)
        driver.find_element(By.ID, value = 'session_password').send_keys(password)
        sleep(1)
        driver.find_element(By.CSS_SELECTOR, value = 'button[data-id = "sign-in-form__submit-btn"').click()
        sleep(2)

        #Accedemos a los empleos
        empleos = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span[title = "Empleos"]')))
        empleos.click()
        sleep(2)

        #Buscamos empleo
        buscador = driver.find_element(By.CSS_SELECTOR, value = 'input[class = "jobs-search-box__text-input jobs-search-box__keyboard-text-input"]')
        buscador.send_keys(empleo)
        sleep(1)
        buscador.send_keys(Keys.ENTER)
        sleep(2)

        #Pulsamos todos los filtros
        todos_los_filtros = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[class = "relative mr2"]')))
        todos_los_filtros.click()

        # Localizamos las fechas de publicacion de los empleos

        fechas_publicacion = [filtro for filtro in driver.find_elements(By.CLASS_NAME, value = 'search-reusables__secondary-filters-filter') if 'Fecha de publicación' in filtro.text][0]
        opciones_fechas_publicacion = fechas_publicacion.text.split('\n')
        opciones_fechas_publicacion = [opcion_fechas_publicacion for opcion_fechas_publicacion in opciones_fechas_publicacion if '«' not in opcion_fechas_publicacion][1:]



        # Obtener el valor seleccionado por el usuario

        publicaciones_seleccionadas = [publicaciones_seleccionadas for publicaciones_seleccionadas in fechas_publicacion.find_elements(By.TAG_NAME, 'li') if intervalo_temporal_busqueda in publicaciones_seleccionadas.text][0]
        publicaciones_seleccionadas.find_element(By.TAG_NAME, 'label').click()

        # Extraemos los tipos de empleos disponibles para la busqueda
        tipos_empleos = [filtro.text for filtro in driver.find_elements(By.CLASS_NAME, value = 'search-reusables__secondary-filters-filter') if 'Tipo de empleo' in filtro.text][0].split('\n')
        tipos_empleos = [tipo for tipo in tipos_empleos if '«' not in tipo][1:]

        # Extraemos los tipos_presencialidad disponibles para la busqueda
        tipos_presencialidad = [filtro.text for filtro in driver.find_elements(By.CLASS_NAME, value = 'search-reusables__secondary-filters-filter') if 'En remoto' in filtro.text][0].split('\n')
        tipos_presencialidad = [presencialidad for presencialidad in tipos_presencialidad if '«' not in presencialidad][1:]

        sleep(1)
        #Cerramos los filtros
        cerrar_filtros = [elemento for elemento in driver.find_elements(By.CSS_SELECTOR, 'span[class = "a11y-text"]') if 'Todos los filtros' in elemento.text][0]
        cerrar_filtros.find_element(By.XPATH, ".//following-sibling::button").click()
    
    except:
        driver.quit()
        print(f'{empleo} no scrapeado')
        continue
    
    scrapear_linkedin(empleo)

    sleep(360)

Desarrollador de software no scrapeado


In [ ]:
sleep(600)

In [ ]:
for empleo in empleos_ingles:
    try:
        driver = webdriver.Chrome()
        driver.get(url)
        sleep(1)
        driver.maximize_window()

        #Iniciamos sesion
        driver.find_element(By.ID, value = 'session_key').send_keys(usuario)
        sleep(1)
        driver.find_element(By.ID, value = 'session_password').send_keys(password)
        sleep(1)
        driver.find_element(By.CSS_SELECTOR, value = 'button[data-id = "sign-in-form__submit-btn"').click()
        sleep(2)

        #Accedemos a los empleos
        empleos = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'span[title = "Empleos"]')))
        empleos.click()
        sleep(2)

        #Buscamos empleo
        buscador = driver.find_element(By.CSS_SELECTOR, value = 'input[class = "jobs-search-box__text-input jobs-search-box__keyboard-text-input"]')
        buscador.send_keys(empleo)
        sleep(1)
        buscador.send_keys(Keys.ENTER)
        sleep(2)

        #Pulsamos todos los filtros
        todos_los_filtros = WebDriverWait(driver, 120).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[class = "relative mr2"]')))
        todos_los_filtros.click()

        # Localizamos las fechas de publicacion de los empleos

        fechas_publicacion = [filtro for filtro in driver.find_elements(By.CLASS_NAME, value = 'search-reusables__secondary-filters-filter') if 'Fecha de publicación' in filtro.text][0]
        opciones_fechas_publicacion = fechas_publicacion.text.split('\n')
        opciones_fechas_publicacion = [opcion_fechas_publicacion for opcion_fechas_publicacion in opciones_fechas_publicacion if '«' not in opcion_fechas_publicacion][1:]



        # Obtener el valor seleccionado por el usuario

        publicaciones_seleccionadas = [publicaciones_seleccionadas for publicaciones_seleccionadas in fechas_publicacion.find_elements(By.TAG_NAME, 'li') if intervalo_temporal_busqueda in publicaciones_seleccionadas.text][0]
        publicaciones_seleccionadas.find_element(By.TAG_NAME, 'label').click()

        # Extraemos los tipos de empleos disponibles para la busqueda
        tipos_empleos = [filtro.text for filtro in driver.find_elements(By.CLASS_NAME, value = 'search-reusables__secondary-filters-filter') if 'Tipo de empleo' in filtro.text][0].split('\n')
        tipos_empleos = [tipo for tipo in tipos_empleos if '«' not in tipo][1:]

        # Extraemos los tipos_presencialidad disponibles para la busqueda
        tipos_presencialidad = [filtro.text for filtro in driver.find_elements(By.CLASS_NAME, value = 'search-reusables__secondary-filters-filter') if 'En remoto' in filtro.text][0].split('\n')
        tipos_presencialidad = [presencialidad for presencialidad in tipos_presencialidad if '«' not in presencialidad][1:]

        sleep(1)
        #Cerramos los filtros
        cerrar_filtros = [elemento for elemento in driver.find_elements(By.CSS_SELECTOR, 'span[class = "a11y-text"]') if 'Todos los filtros' in elemento.text][0]
        cerrar_filtros.find_element(By.XPATH, ".//following-sibling::button").click()

    
    except:
        driver.quit()
        print(f'{empleo} no scrapeado')
        continue
    
    scrapear_linkedin(empleo)

    sleep(360)

In [ ]:
# Actualizamos el fichero de control de extraccion de datos
#actualizar_fichero_control_extracciones_datos(portal_busqueda, fecha_scrapeo)